In [1]:
pip install --upgrade gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:

import pandas as pd
from transformers import pipeline

# Load the CSV containing articles' data
df_articles = pd.read_csv('american_revolution_articles.csv')

# Initialize the Hugging Face question-answering pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Function to retrieve content from the DataFrame
def get_article_content(page_id):
    # Filter the DataFrame for the specific page ID and retrieve the content
    content = df_articles.loc[df_articles['Page ID'] == page_id, 'Content'].iloc[0]
    return content

# Function to get answers to questions based on the content of a specific article
def get_answer(question, page_id):
    # Retrieve content for the specific article
    context = get_article_content(page_id)
    if context:
        # Use the QA model to get an answer
        answer = qa_pipeline(question=question, context=context)
        return answer['answer']
    else:
        return "No relevant content available."
        
# def combined_function(question, page_id):
#     context = get_article_content(page_id)
#     if context:
#         answer = qa_pipeline (question=question, context=context)
#         return answer['answer']
#     else:
#         return "No relevant content available."
#     content = df_articles.loc[df_articles['Page ID'] == page_id, 'Content'].iloc[0]
#     return content
# Example Usage
page_id_example = 82254  # Example page ID from your CSV
question_example = "What should I know about the american revolution?"

# Get the answer
answer = get_answer(question_example, page_id_example)
print(f"Q: {question_example}\nA: {answer}")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Q: What should I know about the american revolution?
A: Depictions, reports, and propaganda about the event heightened tensions throughout the Thirteen Colonies


In [3]:
def combined_function(question, page_id):
    return get_answer(question, page_id) 

In [8]:
import gradio as gr

theme = gr.themes.Default(
    primary_hue="blue",
    neutral_hue="stone",
    
)

# with gr.Blocks(theme=theme) as demo:
# Building the interface functions
question_input = gr.Textbox(label="Question", placeholder="Enter your question here...") 
page_id_input = gr.Textbox(label="Page ID", placeholder="Enter the page ID here...") 
answer_output = gr.Textbox(label="Answer", placeholder="Answer will appear here...")
interface = gr.Interface(fn=combined_function,
                        inputs=[gr.Textbox(placeholder="What"),page_id_input],
                        outputs=gr.Textbox(placeholder=""),
               # theme=theme
               )
                   
interface.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
